In [1]:
from pymongo import MongoClient
import plotly.express as px
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
mongo = MongoClient(port=27017)

In [3]:
print(mongo.list_database_names())

['Spotify_2023', 'admin', 'autosaurus', 'classDB', 'config', 'local', 'met', 'uk_food']


In [4]:
db = mongo['Spotify_2023']


In [5]:
print(db.list_collection_names())

['Song_list']


In [6]:
Most_Popular_Songs_2023 = db['Song_list']

In [7]:
Most_Popular_Songs_2023.find_one()

{'_id': ObjectId('666b8730480d52d219eeeabd'),
 'option': 'Seven (feat. Latto) (Explicit Ver.)',
 'artist(s)_name': 'Latto, Jung Kook',
 'artist_count': 2,
 'released_year': 2023,
 'released_month': 7,
 'released_day': 14,
 'in_spotify_playlists': 553,
 'in_spotify_charts': 147,
 'streams': 141381703,
 'in_apple_playlists': 43,
 'in_apple_charts': 263,
 'in_deezer_playlists': 45,
 'in_deezer_charts': 10,
 'in_shazam_charts': 826,
 'bpm': 125,
 'key': 'B',
 'mode': 'Major',
 'danceability_%': 80,
 'valence_%': 89,
 'energy_%': 83,
 'acousticness_%': 31,
 'instrumentalness_%': 0,
 'liveness_%': 8,
 'speechiness_%': 4}

In [8]:
x = list(Most_Popular_Songs_2023.find().sort({'in_spotify_charts':-1}))


In [9]:
df_Spotify = pd.DataFrame(x)
df_Spotify = df_Spotify.rename(columns={'option':'song_name'})
df_Spotify.head()

,_id,song_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,666b8730480d52d219eeeabd,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,...,125,B,Major,80,89,83,31,0,8,4
1,666b8730480d52d219eeeacb,As It Was,Harry Styles,1,2022,3,31,23575,130,2513188493,...,174,F#,Minor,52,66,73,34,0,31,6
2,666b8730480d52d219eeeac9,Flowers,Miley Cyrus,1,2023,1,12,12211,115,1316855716,...,118,NaN,Major,71,65,68,6,0,3,7
3,666b8730480d52d219eeeabf,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,...,138,F,Major,51,32,53,17,0,31,6
4,666b8730480d52d219eeead3,I Wanna Be Yours,Arctic Monkeys,1,2013,1,1,12859,110,1297026226,...,135,NaN,Minor,48,44,42,12,2,11,3


In [10]:
# Find the row number of the song with inaccurate information
correction_value1 = "Riptide"
row_number1 = df_Spotify.index[df_Spotify['song_name'] == correction_value1].tolist()
row_number1

[39]

In [11]:
df_Spotify.iat[39, 3] = 2013
df_Spotify.iat[39, 3]

2013

In [12]:
# Find the row number of the song with inaccurate information
correction_value2 = "Love Grows (Where My Rosemary Goes)"
row_number2 = df_Spotify.index[df_Spotify['song_name'] == correction_value2].tolist()
row_number2

[703]

In [13]:
df_Spotify.iat[703, 8] = 255230973
df_Spotify.iat[703, 8]

255230973

In [14]:
# Find the row number of the song with inaccurate information
correction_value3 = "Styrx, utku INC, Thezth"
row_number3 = df_Spotify.index[df_Spotify['artist(s)_name'] == correction_value1].tolist()
row_number3

[]

In [15]:
df_Spotify.iat[603, 8] = 2022
df_Spotify.iat[603, 8]

2022

In [16]:
df_Spotify1 = df_Spotify.drop(columns=["danceability_%","valence_%","energy_%","acousticness_%","instrumentalness_%","liveness_%","speechiness_%","_id", "in_apple_playlists", "in_apple_charts", "in_deezer_playlists", "in_deezer_charts", "in_shazam_charts"])
df_Spotify1.head()

,song_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,bpm,key,mode
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,125,B,Major
1,As It Was,Harry Styles,1,2022,3,31,23575,130,2513188493,174,F#,Minor
2,Flowers,Miley Cyrus,1,2023,1,12,12211,115,1316855716,118,NaN,Major
3,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,138,F,Major
4,I Wanna Be Yours,Arctic Monkeys,1,2013,1,1,12859,110,1297026226,135,NaN,Minor


In [42]:
df_Spotify_2 = df_Spotify.head(10)

In [43]:
fig = px.bar(df_Spotify_2, x='song_name', barmode='group', title='Song Popularity Metrics')

# Update the layout to add dropdown menu for switching between different criteria
fig.update_layout(
    yaxis = {'title':'score'},
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Danceability',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['danceability_%']], 'yaxis':{'title':'new test'},'x': [df_Spotify_2['song_name']], 'title': 'danceability_%', 'ylabel':'text'}]
                },
                {
                    'label': 'Valence',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['valence_%']], 'x': [df_Spotify_2['song_name']], 'title': 'valence_%'}]
                },
                {
                    'label': 'Energy',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['energy_%']], 'x': [df_Spotify_2['song_name']], 'title': 'energy_%'}]
                },
                {
                    'label': 'Acousticness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['acousticness_%']], 'x': [df_Spotify_2['song_name']], 'title': 'acousticness_%'}]
                },
                {
                    'label': 'Instrumentalness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['instrumentalness_%']], 'x': [df_Spotify_2['song_name']], 'title': 'instrumentalness_%'}]
                },
                {
                    'label': 'Liveness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['liveness_%']], 'x': [df_Spotify_2['song_name']], 'title': 'liveness_%'}]
                },
                {
                    'label': 'Speechiness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify_2['speechiness_%']], 'x': [df_Spotify_2['song_name']], 'title': 'speechiness_%'}]
                }
            ]
        }
    ]
)
fig.show()

In [24]:

fig = px.bar(df_Spotify, x='in_spotify_charts', y='danceability_%', barmode='group', title='Song Popularity Metrics')

# Update the layout to add dropdown menu for switching between different criteria
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'label': 'Danceability',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['danceability_%']], 'x': [df_Spotify['song_name']], 'title': 'danceability_%'}]
                },
                {
                    'label': 'Valence',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['valence_%']], 'x': [df_Spotify['song_name']], 'title': 'valence_%'}]
                },
                {
                    'label': 'Energy',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['energy_%']], 'x': [df_Spotify['song_name']], 'title': 'energy_%'}]
                },
                {
                    'label': 'Acousticness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['acousticness_%']], 'x': [df_Spotify['song_name']], 'title': 'acousticness_%'}]
                },
                {
                    'label': 'Instrumentalness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['instrumentalness_%']], 'x': [df_Spotify['song_name']], 'title': 'instrumentalness_%'}]
                },
                {
                    'label': 'Liveness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['liveness_%']], 'x': [df_Spotify['song_name']], 'title': 'liveness_%'}]
                },
                {
                    'label': 'Speechiness',
                    'method': 'update',
                    'args': [{'y': [df_Spotify['speechiness_%']], 'x': [df_Spotify['song_name']], 'title': 'speechiness_%'}]
                }
            ]
        }
    ]
)
fig.show()

In [18]:
#x_axis = np.arange(len(df_Spotify1))
#tick_locations = [value for value in x_axis]
#plt.bar(x_axis, df_Spotify1["artist(s)_name"], color='r', alpha=0.5, align="center")
#plt.xticks(tick_locations, df_Spotify1["in_spotify_playlists"], rotation="vertical")
#plt.show()